<a href="https://colab.research.google.com/github/sam-thurman/asl_alphabet_image_classification/blob/master/exploratory_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import scipy as math
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
# from PIL import Image

from sklearn.model_selection import train_test_split
import keras

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras.models import Model, Sequential
from keras import layers
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras import optimizers
from keras.optimizers import Adam

import sys
import os


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [3]:
# mount google drive
from google.colab import drive
# drive.mount('/content/drive/My Drive/data')
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
# change directory into drive
%cd drive/My\ Drive/
%ls

/content/drive/My Drive
 Accounts.gdoc
 asl_alphabet_image_classification/
'bb logo v2 (05:29:19).pdf'
'Behavioral quizz write-up 301.gdoc'
'Capstone MVP presentation.gslides'
'ceative resume.gdoc'
'CF 301 reading journal.gdoc'
 Codefellows/
'(codefellows)Answers to Common Behavioral Questions.gdoc'
 communication.mp3
 copy_base__inceptionv3.h5
 copy_base_model.h5
 copy_model1.h5
 copy_model2.h5
 data/
 Fin:BobbyBrooka_AsIShould.mp3
 Fin_Mischief.mp3
'Fin_Wreckage 3.mp3'
'Flatiron Capstone Proposal.gdoc'
'flatiron notes'/
'Getting started.pdf'
"hot n' cold beat RW.mp3"
'ice man (concept not finished).mp3'
'Image 11-18-19 at 11.12 AM.JPG'
'in my ceiling.mp3'
'in my ceiling (w VERY ROUGH voc concept).mp3'
'Kaia_RedWine_BetweenUs 2.mp3'
 Kaia_RedWine_BetweenUs.mp3
'mischief (1).mp3'
 mischief.mp3
'Norwood_TollFree[Master].mp3'
'nothing nowhere.mp3'
'pain on your thigh'/
'pain on your thigh 1-Drum Rack.mp3'
'pain on your thigh 2-Addictive Keys.mp3'
'pain on your thigh 3-Addictive Keys.mp3'

In [0]:
train_path = 'data/asl_alphabet_train'
validation_path = 'data/asl_alphabet_validation'

In [0]:
# image generator instance
datagen = keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255,
                validation_split=0.3) 
valgen = keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255) 

In [8]:

image_size = 128
batch_size = 32

print('done')
train_generator = datagen.flow_from_directory(
                train_path,  #directory for training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='categorical',
                color_mode='grayscale',
                shuffle=True,
                subset='training')
print('done')
test_generator = datagen.flow_from_directory(
                train_path,  # same directory for testing images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='categorical',
                color_mode='grayscale',
                shuffle=True,
                subset='validation')
print('done')
val_generator = valgen.flow_from_directory(
                validation_path,  # directory for validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='categorical',
                color_mode='grayscale',
                shuffle=False)
print('done')

done
Found 60991 images belonging to 29 classes.
done
Found 26134 images belonging to 29 classes.
done
Found 30 images belonging to 29 classes.
done


In [0]:
nb_classes = 29

# initialize CNN
model3 = Sequential()

# 1st conv layer
model3.add(Conv2D(32,(3,3), padding='same', input_shape=(image_size, image_size,1)))
model3.add(BatchNormalization())
model3.add(Activation('relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Dropout(0.25))

# 2nd conv layer
model3.add(Conv2D(64,(5,5), padding='same'))
model3.add(BatchNormalization())
model3.add(Activation('relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Dropout(0.25))

# 3rd conv layer
model3.add(Conv2D(128,(3,3), padding='same'))
model3.add(BatchNormalization())
model3.add(Activation('relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Dropout(0.25))

# 4th conv layer
model3.add(Conv2D(128,(3,3), padding='same'))
model3.add(BatchNormalization())
model3.add(Activation('relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Dropout(0.25))

# flattening
model3.add(Flatten())

# 1st fully connected layer
model3.add(Dense(128))
model3.add(BatchNormalization())
model3.add(Activation('relu'))
model3.add(Dropout(0.4))

# 2nd fully connected layer
model3.add(Dense(64))
model3.add(BatchNormalization())
model3.add(Activation('relu'))
model3.add(Dropout(0.3))

model3.add(Dense(nb_classes, activation='softmax'))



In [0]:
optimizer = Adam(lr=0.01)
model3.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
hisotry3 = model3.fit_generator(train_generator, epochs=5)

Epoch 1/5
  97/1906 [>.............................] - ETA: 5:21:33 - loss: 3.2665 - accuracy: 0.0892

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 169 could not be retrieved. It could be because a worker has died.
  UserWarning)


 502/1906 [======>.......................] - ETA: 4:02:12 - loss: 2.5110 - accuracy: 0.2399

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 399 could not be retrieved. It could be because a worker has died.
  UserWarning)


1217/1906 [==================>...........] - ETA: 1:57:13 - loss: 1.8402 - accuracy: 0.4176

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 623 could not be retrieved. It could be because a worker has died.
  UserWarning)


1576/1906 [=======================>......] - ETA: 55:36 - loss: 1.6471 - accuracy: 0.4733

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 1048 could not be retrieved. It could be because a worker has died.
  UserWarning)


1906/1906 [==============================] - 19167s 10s/step - loss: 1.5075 - accuracy: 0.5150
Epoch 2/5
1906/1906 [==============================] - 689s 361ms/step - loss: 0.6039 - accuracy: 0.7954
Epoch 3/5
1906/1906 [==============================] - 690s 362ms/step - loss: 0.3793 - accuracy: 0.8743
Epoch 4/5
1906/1906 [==============================] - 693s 364ms/step - loss: 0.2756 - accuracy: 0.9112
Epoch 5/5
1906/1906 [==============================] - 691s 362ms/step - loss: 0.2221 - accuracy: 0.9278


In [0]:
model3.save('model3.h5')
del model3

In [14]:
%ls

 Accounts.gdoc
 asl_alphabet_image_classification/
'bb logo v2 (05:29:19).pdf'
'Behavioral quizz write-up 301.gdoc'
'Capstone MVP presentation.gslides'
'ceative resume.gdoc'
'CF 301 reading journal.gdoc'
 Codefellows/
'(codefellows)Answers to Common Behavioral Questions.gdoc'
 communication.mp3
 copy_base__inceptionv3.h5
 copy_base_model.h5
 copy_model1.h5
 copy_model2.h5
 data/
 Fin:BobbyBrooka_AsIShould.mp3
 Fin_Mischief.mp3
'Fin_Wreckage 3.mp3'
'Flatiron Capstone Proposal.gdoc'
'flatiron notes'/
'Getting started.pdf'
"hot n' cold beat RW.mp3"
'ice man (concept not finished).mp3'
'Image 11-18-19 at 11.12 AM.JPG'
'in my ceiling.mp3'
'in my ceiling (w VERY ROUGH voc concept).mp3'
'Kaia_RedWine_BetweenUs 2.mp3'
 Kaia_RedWine_BetweenUs.mp3
'mischief (1).mp3'
 mischief.mp3
 model3.h5
'Norwood_TollFree[Master].mp3'
'nothing nowhere.mp3'
'pain on your thigh'/
'pain on your thigh 1-Drum Rack.mp3'
'pain on your thigh 2-Addictive Keys.mp3'
'pain on your thigh 3-Addictive Keys.mp3'
'pain on you

In [0]:
from keras.models import load_model
model3 = load_model('model3.h5')

In [16]:
model3.evaluate_generator(val_generator, verbose=1)

1/1 [==============================] - 9s 9s/step


[0.16693522036075592, 0.9666666388511658]